# Importing packages

In [1]:
# Import packages
import os
import cv2
import numpy as np
import tensorflow as tf
import sys

os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("./models/research/object_detection/")


# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

2023-11-28 05:39:16.197064: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 05:39:16.197242: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 05:39:16.223320: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 05:39:16.280940: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 05:39:17.361327: W tensorflow/compiler/tf2

# Helmet Detection in Image

In [40]:
# Name of the directory containing the object detection module we're using
MODEL_NAME = 'inference_graph'
IMAGE_NAME = 'sample.jpeg'

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

# Path to image
PATH_TO_IMAGE = os.path.join(CWD_PATH,IMAGE_NAME)

# Number of classes the object detector can identify
NUM_CLASSES = 4

# Load the label map.
# Label maps map indices to category names
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
    sess = tf.compat.v1.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

# Load image using OpenCV and
# expand image dimensions to have shape: [1, None, None, 3]
# i.e. a single-column array, where each item in the column has the pixel RGB value
image = cv2.imread(PATH_TO_IMAGE)
# image=cv2.resize(image,(720,480))
image_expanded = np.expand_dims(image, axis=0)

# Perform the actual detection by running the model with the image as input
(boxes, scores, classes, num) = sess.run(
    [detection_boxes, detection_scores, detection_classes, num_detections],
    feed_dict={image_tensor: image_expanded})

print("boxes-->", boxes.shape)
print("scores-->", scores.shape)
print("classes-->", classes.shape)
print("num-->", num)

i=-1

# Draw the results of the detection (aka 'visulaize the results')
indices_to_remove = []

for i, box in enumerate(boxes[0]):
    if classes[0][i] == 4:
        indices_to_remove.append(i)

# Delete the elements based on the collected indices
newboxes = np.delete(boxes, indices_to_remove, axis=1)
newclasses = np.delete(classes, indices_to_remove, axis=1)
newscores = np.delete(scores, indices_to_remove,axis=1)
num[0] -= len(indices_to_remove)

print("boxes-->", newboxes.shape)
print("scores-->", newscores.shape)
print("classes-->", newclasses.shape)
print("num-->", num)

vis_util.visualize_boxes_and_labels_on_image_array(
    image,
    np.squeeze(newboxes),
    np.squeeze(newclasses).astype(np.int32),
    np.squeeze(newscores),
    category_index,
    use_normalized_coordinates=True,
    line_thickness=8,
    min_score_thresh=0.80)

# All the results have been drawn on image. Now display the image.
cv2.imwrite('output.jpg',image)
cv2.imshow('Object detector', image)


# Press any key to close the image
cv2.waitKey(0)

# Clean up
cv2.destroyAllWindows()


labelmap--> item {
  name: "BikeRider"
  id: 1
}
item {
  name: "Helmet"
  id: 2
}
item {
  name: "WithoutHelmet"
  id: 3
}
item {
  name: "LicensePlate"
  id: 4
}

item--> name: "BikeRider"
id: 1

item--> name: "Helmet"
id: 2

item--> name: "WithoutHelmet"
id: 3

item--> name: "LicensePlate"
id: 4



W0000 00:00:1701134090.003112  245346 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "CropAndResize" attr { key: "T" value { type: DT_FLOAT } } attr { key: "extrapolation_value" value { f: 0 } } attr { key: "method" value { s: "bilinear" } } inputs { dtype: DT_FLOAT shape { dim { size: -7 } dim { size: -10 } dim { size: -12 } dim { size: 576 } } } inputs { dtype: DT_FLOAT shape { dim { size: -35 } dim { size: 4 } } } inputs { dtype: DT_INT32 shape { dim { size: -35 } } } inputs { dtype: DT_INT32 shape { dim { size: 2 } } value { dtype: DT_INT32 tensor_shape { dim { size: 2 } } int_val: 14 } } device { type: "CPU" vendor: "AuthenticAMD" model: "240" frequency: 3293 num_cores: 12 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 524288 l3_cache_size: 16777216 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { dim { size: -35 } dim {

boxes--> (1, 300, 4)
scores--> (1, 300)
classes--> (1, 300)
num--> [300.]
boxes--> (1, 205, 4)
scores--> (1, 205)
classes--> (1, 205)
num--> [205.]


In [44]:
image = cv2.imread(PATH_TO_IMAGE)
target_class = 3  # Change this to the class you are interested in
score_threshold = 0.80

# Extract cropped images based on class and score
bikeRiderBox = []
withoutHelmet=[]
for i in range(int (num[0])):
    if newscores[0][i] >= score_threshold and int(newclasses[0][i]) == 1:
        box = newboxes[0][i]
        bikeRiderBox.append(list(box))

    elif newscores[0][i] >= score_threshold and int(newclasses[0][i]) == 3:
        box = newboxes[0][i]
        withoutHelmet.append(list(box))

print("bikeriders-->", bikeRiderBox)
print("withoutHelmet-->", withoutHelmet)
# for i in range(int(num[0])):
#     if newscores[0][i] >= score_threshold and int(newclasses[0][i]) == target_class:
#         box = newboxes[0][i]
        # ymin, xmin, ymax, xmax = box
        # im_height, im_width, _ = image.shape
        # (left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
        # left, right, top, bottom = int(left), int(right), int(top), int(bottom)

        # # Ensure the box coordinates are within the image bounds
        # left = max(0, left)
        # right = min(im_width, right)
        # top = max(0, top)
        # bottom = min(im_height, bottom)

        # # Crop the image based on the box coordinates
        # cropped_image = image[top:bottom, left:right]

        # # Save or process the cropped image as needed
        # cv2.imwrite(f'cropped_image_{i}.jpg', cropped_image)

i=0

for biker in bikeRiderBox:
    
    for noHelmet in withoutHelmet:
        if((biker[0]<noHelmet[0] or abs(biker[0]-noHelmet[0])<0.05) and (biker[1]<noHelmet[1] or abs(biker[1]-noHelmet[1])<0.05) and (biker[2]>noHelmet[2] or abs(biker[2]-noHelmet[2])<0.05) and (biker[3]>noHelmet[3] or abs(biker[3]-noHelmet[3])<0.05)):
            ymin, xmin, ymax, xmax = biker
            im_height, im_width, _ = image.shape
            (left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
            left, right, top, bottom = int(left), int(right), int(top), int(bottom)

            # Ensure the box coordinates are within the image bounds
            left = max(0, left)
            right = min(im_width, right)
            top = max(0, top)
            bottom = min(im_height, bottom)

            # Crop the image based on the box coordinates
            cropped_image = image[top:bottom, left:right]

            # Save or process the cropped image as needed
            cv2.imwrite(f'./temp_img/cropped_image_{i}.jpg', cropped_image)
            i=i+1
            print(i)


bikeriders--> [[0.034997538, 0.8327921, 0.9874186, 0.9955711], [0.08783651, 0.27999228, 1.0, 0.594836], [0.09565405, 0.15384576, 0.7942245, 0.30933025]]
withoutHelmet--> [[0.10323922, 0.20530592, 0.20591743, 0.2573324], [0.099633165, 0.38904864, 0.25493, 0.4636621]]
1
2


# Helmet Detection in video

In [ ]:
# Name of the directory containing the object detection module we're using
MODEL_NAME = 'inference_graph'
VIDEO_NAME = 'testvideo.mkv'

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

# Path to video
PATH_TO_VIDEO = os.path.join(CWD_PATH,VIDEO_NAME)

# Number of classes the object detector can identify
NUM_CLASSES = 4

# Load the label map.
# Label maps map indices to category names
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

# Open video file
video = cv2.VideoCapture(PATH_TO_VIDEO)

while(video.isOpened()):

    # Acquire frame and expand frame dimensions to have shape: [1, None, None, 3]
    # i.e. a single-column array, where each item in the column has the pixel RGB value
    ret, frame = video.read()
    frame=cv2.resize(frame,None,fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    frame_expanded = np.expand_dims(frame, axis=0)

    # Perform the actual detection by running the model with the image as input
    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: frame_expanded})

    # Draw the results of the detection (aka 'visulaize the results')
    vis_util.visualize_boxes_and_labels_on_image_array(
        frame,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8,
        min_score_thresh=0.80)

    # All the results have been drawn on the frame, so it's time to display it.
    cv2.imshow('Object detector', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) == ord('q'):
        break

# Clean up
video.release()
cv2.destroyAllWindows()
